### Load configuration

In [8]:
from JSONReader import read_json
config_path = 'config.json'
config_dict = read_json(config_path)
OMXS30_tickers = read_json('omxs30.json')
print(f"Configuration: {config_dict}")
print(f"Tickers: {OMXS30_tickers}")

import torch
device = torch.device(
    'cuda' 
    if torch.cuda.is_available()
    else 'mps' 
    if torch.backends.mps.is_available() 
    else 'cpu')
print(f"Device: {device}")

Configuration: {'seq_len': 60, 'pred_len': 12, 'train_start': '2020-01-01', 'train_end': '2022-12-31', 'test_start': '2023-01-01', 'test_end': '2023-12-31', 'hidden': 500, 'n_lstm': 3, 'batch_size': 3, 'lr': 0.0003, 'epochs': 10}
Tickers: ['ABB.ST', 'ALFA.ST', 'ASSA-B.ST', 'ATCO-A.ST', 'ATCO-B.ST', 'AZN.ST', 'BOL.ST', 'ELUX-B.ST', 'ERIC-B.ST', 'ESSITY-B.ST', 'EVO.ST', 'GETI-B.ST', 'HEXA-B.ST', 'HM-B.ST', 'INVE-B.ST', 'KINV-B.ST', 'NDA-SE.ST', 'NIBE-B.ST', 'SAAB-B.ST', 'SAND.ST', 'SBB-B.ST', 'SCA-B.ST', 'SEB-A.ST', 'SHB-A.ST', 'SINCH.ST', 'SKF-B.ST', 'SWED-A.ST', 'TEL2-B.ST', 'TELIA.ST', 'VOLV-B.ST']
Device: mps


### Load train and test datasets

In [9]:
from torch.utils.data import DataLoader
from dataset_def import ReturnDataset
import datetime
seq_len = config_dict['seq_len']
pred_len = config_dict['pred_len']
batch_size = config_dict['batch_size']

train_start = datetime.datetime.strptime(config_dict['train_start'],'%Y-%m-%d')
train_end = datetime.datetime.strptime(config_dict['train_end'],'%Y-%m-%d')

train_dataset = ReturnDataset(
    seq_len=seq_len,
    prediction_len=pred_len,
    ticker_list=OMXS30_tickers ,
    start_date=train_start,
    end_date=train_end)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size,shuffle=False) #data is already shuffled

test_start = datetime.datetime.strptime(config_dict['test_start'],'%Y-%m-%d')
test_end = datetime.datetime.strptime(config_dict['test_end'],'%Y-%m-%d')

test_dataset = ReturnDataset(seq_len=seq_len,
                             prediction_len=pred_len,
                             ticker_list=OMXS30_tickers,
                             start_date=test_start,
                             end_date=test_end)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size,shuffle=False) #data is already shuffled



In [10]:
print(f"Sequence length: {seq_len}")
print(f"Prediction length: {pred_len}")
print(f"Train start-date: {config_dict['train_start']}")
print(f"Train end-date: {config_dict['train_end']}")
print(f"Test start-date: {config_dict['test_start']}")
print(f"Test end-date: {config_dict['test_end']}")

Sequence length: 60
Prediction length: 12
Train start-date: 2020-01-01
Train end-date: 2022-12-31
Test start-date: 2023-01-01
Test end-date: 2023-12-31


### Initiate LSTM model for return predictions

In [11]:
from lstm_predictor import LSTMPredictor

input_size = train_dataset.num_feats
hidden_size = config_dict['hidden']
output_size = pred_len
n_layers = config_dict['n_lstm']

model = LSTMPredictor(
    input_size=input_size,
    hidden_size=hidden_size,
    output_size=output_size,
    n_layers=n_layers,
    batch_first=True,
)
model.to(device=device)

LSTMPredictor(
  (lstm1): LSTM(6, 500, num_layers=3, batch_first=True)
  (activation): SiLU()
  (fco): Linear(in_features=500, out_features=12, bias=True)
)

### Train model using LSTMTrainer

In [13]:
from lstm_trainer import LSTMTrainer

trainer = LSTMTrainer(
    model=model,
    train_dataloader=train_dataloader,
    test_dataloader=test_dataloader,
    config_dict=config_dict,
    train_scaler_dict=train_dataset.scaler_dict,
    test_scaler_dict=test_dataset.scaler_dict,
    report_freq=100,
    device=device,
)
train_batch_loss, test_batch_loss = trainer.full_epoch_loop()

---------- Full epoch: 1 ----------
---------- Full epoch: 2 ----------
---------- Full epoch: 3 ----------
---------- Full epoch: 4 ----------
---------- Full epoch: 5 ----------
---------- Full epoch: 6 ----------
---------- Full epoch: 7 ----------
---------- Full epoch: 8 ----------
---------- Full epoch: 9 ----------
---------- Full epoch: 10 ----------


### Visualize model and results

In [14]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()
graph_batch,_,_ = next(iter(train_dataloader))
if device == torch.device('mps'):
    graph_batch_dtype = torch.float32
else:
    graph_batch_dtype = torch.float64

writer.add_graph(model=model, input_to_model=graph_batch.to(device=device,dtype=graph_batch_dtype))
mean_epoch_array = train_batch_loss.mean(1)
for i in range(train_batch_loss.shape[0]):
    writer.add_scalar('train_mean_epoch_loss',scalar_value=mean_epoch_array[i],global_step=i)
